# Introduction to Julia
Tommy Hofmann (University of Siegen) — 6/9/2021

## What is Julia?
- Created in 2012 at MIT
- Imperative with syntax similar to python
- Performance similar to C
- Dynamic, strictly typed lanuage
- Not object oriented (no OOP)
- Interactive

## Why Julia?
- Because your adivsor says so

## What will we learn today?
1. Control flow
2. Basic types
3. Functions and methods
4. More on types and dispatch
5. Gotchas
6. Demo

## Control flow

### Conditional statements (if/else)

In [1]:
x = 2

if x > 0
    y = 1
elseif x < 1
    y = -1
else
    y = 0
end

print("y is $y")

y is 1

### Ternary conditional operator
(Same as Magmas "select")

In [2]:
x = 2

if x > 0
     y = 1
else
     y = -1
end

println("y is $y")

y = x > 0 ? 1 : -1

println("y is $y")

y is 1
y is 1


### For loops
We look at the following example of adding all odd entries in an array `A`

In [3]:
A = [1,2,3,4,5,6,7,8,9,10] # a list of type Vector
accum = 0
for i in A
    if i % 2 != 0
        accum += i
    end
end
accum

25

We can also write this as follows:

In [4]:
A = 1:10
accum = 0
for i in A
    if i % 2 != 0
        accum += i
    end
end
accum

25

### Range objects

Note that `1:10` is a range object, which represents the list `1,2,...,10`. Ranges come in different flavors. If one really needs the array of the elements in the range, one can use `collect`.

In [1]:
x = 1:10
println(x)
println(collect(x))
y = 1:2:10
println(collect(y))
z = 10:-1:1
println(collect(z))

1:10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 3, 5, 7, 9]
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


### While loops

In [3]:
i = 1
accum = 0
while i <= 10
    accum += i
    i += 1
    if accum > 40
        println("Too large!")
        break
    end
    
    if i == 2
        println("i is $i")
        continue
    end
end
accum

i is 2
Too large!


45

## Basic Julia types

### `Int` (aka `Int64`)
Values of this type are usually called machine integers, which is roughly equivalent to working modulo $2^{64}$.

In [7]:
x = 1
typeof(x)

Int64

In [8]:
x = 2^64

0

In [9]:
Int === Int64

true

`Int` should not be used when doing computations in $\mathbb{Z}$, but for bookkeeping, indexing and control flow.

### `Float64`

These are classical floating point numbers (with 53 bits of precision), which are also known as `double`.

In [10]:
x = 1.0
typeof(x)

Float64

### `String`

In [11]:
x = "a + b"
typeof(x)

String

There is something called string interpolation, which is quite useful when quickly printing some things:

In [12]:
x = 3 # this is an Int
println("x is of type $(typeof(x)) with value $(x)")

x is of type Int64 with value 3


### `BigInt`

These is integers with arbitrary size.

In [13]:
x = BigInt(2)^100

1267650600228229401496703205376

### `Vector`
This is the basic types for lists. Note that the fullname is `Vector{T}`, where `T` is the type of the elements of the list. Here are some useful ways to construct lists.

In [14]:
x = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [15]:
v = Vector{Int}(undef, 3) # Create a list for values of type `Int` with length 3. The entries in `v` are undefined.
v[1] = 1; v[2] = -1; v[3] = 4
v

3-element Vector{Int64}:
  1
 -1
  4

__Warning__: The name is quite misleading! We should not think of these as vectors (with mathematical structure), but as lists.

### `Matrix`
This is the basic type for 2-dimensional arrays. As for `Vector`, it actually is `Matrix{T}`, where `T` is the type of the elements of the list. Here are some constructors:

In [16]:
x = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [17]:
v = Matrix{Int}(undef, 2, 3)
v[1, 1] = 1; v[1, 2] = 2; v[1, 3] = 3; v[2, 1] = 4; v[2, 2] = 5; v[2, 3] = 6;
v

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

### `Array`
These are multidimensional arrays, have full type `Array{T, N}`, where `T` is the type of the elements and `N` is the dimension. In fact, by definition we have `Vector{T} === Array{T, 1}` and `Matrix{T} === Array{T, 2}`.

Of course, all of these constructions can be nested:

In [18]:
x = [[1, 2], [3, 4]]

2-element Vector{Vector{Int64}}:
 [1, 2]
 [3, 4]

### Tuples

Tuples are lists of fixed length in which each entry has its own defined type.

In [19]:
a = (1, "foo")
typeof(a)

Tuple{Int64, String}

They are immutable:

In [20]:
a = (1, "foot")
println(a[1])
a[1] = 2

1


LoadError: MethodError: no method matching setindex!(::Tuple{Int64, String}, ::Int64, ::Int64)

### Dictionaries/Associative arrays/Maps
A dictionary is a data structure for storing objects in a list, where the indices are arbitrary objects. One can think of it as storing values of a map $f \colon D \to C$.

In [3]:
D = Dict("a" => 1, "b" => 2)

Dict{String, Int64} with 2 entries:
  "b" => 2
  "a" => 1

In [4]:
D["a"]

1

In [5]:
println(haskey(D, "d"))
D["d"] = 5
println(haskey(D, "d"))

false
true


In [24]:
D

Dict{String, Int64} with 3 entries:
  "b" => 2
  "a" => 1
  "d" => 5

### Generation of data structures

#### List comprehension:

In [5]:
g(x) = x^2
[g(i) for i in 1:5]

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [6]:
[g(i + j^2) for i in 1:3, j in 1:3]

3×3 Matrix{Int64}:
  4  25  100
  9  36  121
 16  49  144

In [7]:
[g(i) for i in 1:5 if i % 2 == 0]

2-element Vector{Int64}:
  4
 16

In [8]:
[g(i + j) for i in 1:3 for j in 1:3]

9-element Vector{Int64}:
  4
  9
 16
  9
 16
 25
 16
 25
 36

Also for dictionaries:

In [28]:
D = Dict(i => g(i) for i in 1:3)

Dict{Int64, Int64} with 3 entries:
  2 => 4
  3 => 9
  1 => 1

#### Broadcasting:

In [29]:
a = [-2, -1, 0, 1, 2]
g.(a) # this is the same as [g(i) for i in a]

5-element Vector{Int64}:
 4
 1
 0
 1
 4

# Functions and methods

In [30]:
function oddsum(A)
    accum = 0
    for i in A
        if i % 2 != 0
            accum += i
        end
    end
    return accum
end

oddsum (generic function with 1 method)

In [31]:
oddsum(1:10)

25

In [32]:
oddsum([1,2,3,4,5,6,7,8,9,10])

25

## Methods and multiple dispatch

One of the must useful features of Julia is the ability to define multiple methods for the same function combined with multiple dispatch.

In [33]:
foo(x::Int) = x
foo(x::Vector{Int}) = x[1]

foo (generic function with 2 methods)

In [34]:
foo(1)

1

In [35]:
foo([2,3,1])

2

In [36]:
foo(1.0)

LoadError: MethodError: no method matching foo(::Float64)
[0mClosest candidates are:
[0m  foo([91m::Int64[39m) at In[33]:1
[0m  foo([91m::Vector{Int64}[39m) at In[33]:2

This gives a `MethodError`, which tells us that Julia knows of no method of `foo` with accepts a `Float64`. To help us, Julia will print methods of `foo` that are known and why they don't match with our required arguments.

### Dispatch and abstract types
Dispatch is used heavily with *abstract* types. We have for example the following subtype relations:


In [37]:
Int <: Real <: Any

true

The type `Int` is a subtype of `Real`, which is a subtype of `Any`.

In [9]:
function bar(x) # same as bar(x::Any)
    return x + 1
end

function bar(x::Real)
    return x + 2
end

function bar(x::Int)
    return x + 3
end

bar(1 * im), bar(1.0), bar(1) # typeof(1*im) == Complex{Int} <: Any

(1 + 1im, 3.0, 4)

## Custom types

In [39]:
abstract type Shape end

In [40]:
struct Circle <: Shape
    r::Float64
end

struct Square <: Shape
    s::Float64
end

In [41]:
C = Circle(1.0)

Circle(1.0)

In [42]:
function area(C::Circle)
    return pi * C.r^2
end

function area(S::Square)
    return S.s^2
end

area (generic function with 2 methods)

In [43]:
area(Circle(1.0))

3.141592653589793

Let's define an ordering on the shapes given by the area. It is sufficient to define a method for `Base.isless`, which automatically defines all other kind of functions.

In [44]:
Base.isless(A::Shape, B::Shape) = area(A) < area(B)

In [45]:
Circle(1.0) < Square(1.0)

false

In [46]:
sort([Circle(1.0), Square(1.0), Square(2.0)])

3-element Vector{Shape}:
 Square(1.0)
 Circle(1.0)
 Square(2.0)

## Some gotchas and tipps:
- Typing `1/2` will produce `0.5`, which is a floating point number. The correct way is to use `//`, so `1//2` or `x//(x + 1)` rational numbers or a rational function.
- Type `?bla` to get some information about the function `bla`.
- Use tab completion:
  - `gcd\tab` will print all functions whose name start with `gcd`. (This also works with variables in a session.)
  - `f(a, \tab` will print the methods that work with `a` as the first argument for `f` .
  - Sometimes methods are not exported, so one has to prefix them, like `Oscar.isprime_power`.
- Use `@show f` to do some quick debugging.

## Ressources

- Wikibooks on learning Julia: https://en.wikibooks.org/wiki/Introducing_Julia
- Other tutorials: https://julialang.org/learning/tutorials/
- The Julia manual: https://docs.julialang.org/en/v1/

# Demo time!

### Introspection

Having a function with multiple methods is quite powerful, but sometimes makes it hard to figure out which method is actually called. Luckily Julia has some very useful tool for type introspection, namely the macro `@which` (we won't discuss what a macro actually is for now).

In [47]:
function f(x::Number)
   return x^2
end

function f(x::Integer)
   return x + 1
end

f (generic function with 2 methods)

In [48]:
@which f(2.0)

f(x::Number) in Main at In[47]:1

In [49]:
@which f(1)

f(x::Integer) in Main at In[47]:5

This works for almost any function:

In [50]:
@which 1 + 1

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87

In [51]:
@which push!([1, 2], 2)

push!(a::Vector{T}, item) where T in Base at array.jl:926

A similar macro is `@less`, which actually shows the source code of the method that will be called.

## Possible Julia workflows (Demo time!)

### Use vscode and use the play button.

### Use `$EDITOR` and Revise.

Assume that one is working on a file `test.jl`. One usually has a Julia session open and then iterates the following steps:

1. Make changes to `test.jl`.
2. run `include("test.jl")` in the Julia session.
3. Test if the changes lead to the desired outcome.

This works quite well, but of course it does not scale. What if I have more than one file? Also it does not work when working directly on Hecke/Oscar?

This is where the package "Revise" comes into play, which tracks local file changes and automatically loads functions/files that have changed. This basically eliminates step 2. First step is to install Revise using the package manager by invoking `]add Revise`. Once this is done, we load Revise and tell it to track our file "test.jl"

```julia
using Revise
Revise.includet("test.jl")
```

Julia development without Revise is very cumbersome! Always use Revise!